In [ ]:
import pickle
import pandas
import json
import networkx as nx
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm

 ## AF patients 

In [ ]:
def graph_from_onto(onto):
    G = nx.DiGraph()
    cl_edges = []
    for s, ts in onto.items():
        for t in ts:
            cl_edges.append((s.replace('S-', ''), t.replace('S-', '')))
    G.add_edges_from(cl_edges)
    return G

def load_onto(o):
    with open(f'/projects/data/GS/{o}', 'rb') as f:
        onto = pickle.load(f)
    G = graph_from_onto(onto)
    return G

def expand_codes(onto, codes):
    expanded = {}
    for name, top_codes in codes.items():
        expanded[name] = set()
        for c in top_codes:
            expanded[name].add(c)
            if c in onto:
                expanded[name].update(nx.ancestors(onto, c))
            else:
                print("NOT FOUND:", c, name)
        print(name, len(top_codes), len(expanded[name]))
    return expanded

In [ ]:
onto = load_onto('isa_rela_ch2pt_202009.pickle')

In [ ]:
af = { 'AF' : ['49436004']}

In [ ]:
af = expand_codes(onto,af)

In [ ]:
af_codes = set()
for x in af.values():
    af_codes.update(x)

In [ ]:
%%time 
with open('/projects/data/GS/pt2cui_pos_dates.pickle', 'rb') as f:
    pt2cui_pos_dates = pickle.load(f)

In [ ]:
# identify patients with at least one mention AF

def build_set(codes_list):
    '''
    this function returns 
    :param codes:
    :return data: 
    '''
    data = set()
    for pt in pt2cui_pos_dates:
        for ptcodes in pt2cui_pos_dates[pt]:
            if ptcodes in codes_list:
                data.add(pt) 
    return data

In [ ]:
af_patients = build_set(af_codes)
len(af_patients)

In [ ]:
af_ids = list(af_patients)
af_ids = pandas.DataFrame(data={'af_ids': af_ids})
af_ids.to_csv('/projects/ROCKET AF/1A/data extraction/af_ids.csv', sep=',',index=False)